In [1]:
%%javascript
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu">Indíce<i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

<IPython.core.display.Javascript object>

In [2]:
%%html
<style>
    h3{
        color:#0094E4;
    }
    h2{
         color:#F28773;
    }
    h1{
         color:#0094E4;
    }
</style>

<img src="./images/logo.png" style="width: 20%; margin-left: 70%;">
<img src="./images/python.png">

# Spark desde cero

Spark es un **general propose data processing engine** (motor de procesamiento de datos de propósito general).
Está optimizado para correr en memoria (rápido).
Ayuda a procesar datos más rápido que otras alternativas de cómputo distribuido (Map reduce de hadoop).


# Generalidades
## ¿Qué es el cómputo distribuido?

Para manejar grandes cantidades de datos tenemos dos opciones:

- Tener una computador muy poderosa y con muchos recursos ($$$)
- Tener varias computadoras sencillas (como las que tenemos en casa) conectadas en un cluster.  

Por lo general vamos a preferir la segunda opción.

Al tener varias computadoras, no tiene sentido poner a trabajar a todas en una misma tarea, sino que lo más eficiente es usar el paradigma divide y vencerás.

<img src="./images/DIVIDE_Y_VENCERAS.jpg" style="width: 70%">

<img src="./images/divide-and-conquer1.png">

http://bigdata.ices.utexas.edu/project/divide-conquer-methods-for-big-data-analytics/

## MapReduce

Spark se basa en un framework que utiliza la aproximación divide y vencerás el cual se llama MapReduce. MapReduce incluye oras cosas además de la idea de dividir un problema en tareas simples y después conjuntar un problema, incluye cosas como:

* Guardado de estados intermedios
* identificación de nodos muertos
* Balance de cargas
* **Etiquetado de resultados para indicar qué resultados van con cuáles.**

En suma, MapReduce se trata de:
1. Dividir el trabajo en serie (similar que divide y vencerás pero con una relación que puede ser de uno a varios: i.e. puede haber más de un nodo del clúster dedicado a una misma tarea y más de una tarea por nodo del clúster).
2. Ejecutar pedazos de trabajo en paralelo (al igual que en la idea de divide y vencerás).
3. **Etiquetado de resultados para indicar qué resultados van con cuáles.**
4. **Merging serializado para todos los resultados que tienen la misma etiqueta y que puede ser en paralelo para distintas etiquetas.**

En python existen las instrucciones **map** y **reduce**. Donde map aplica una función a un arreglo ("distribuye"), mientras que reduce toma una instrucción a aplicarse sobre dos elementos y la aplica a todo un arreglo.

Para ello, primero necesitamos una función (puede ser una lambda).

### map

In [ ]:
def fahrenheit(T):
    return ((float(9)/5)*T + 32)
def celsius(T):
    return (float(5)/9)*(T-32)
temp = (36.5, 37, 37.5,39)

F = map(fahrenheit, temp)
C = map(celsius, F)

In [ ]:
F

In [ ]:
C

In [ ]:
Celsius = [39.2, 36.5, 37.3, 37.8]
Fahrenheit = map(lambda x: (float(9)/5)*x + 32, Celsius)

In [ ]:
print (list(Fahrenheit))

¿Qué hizo map?

### reduce

In [ ]:
import functools
functools.reduce(lambda x,y: x+y, [47,11,42,13])

<img src="./images/reduce_diagram.png">


<img src="./images/MapReduce.jpg" style="width: 70%">
<img src="./images/WordCount_MapReduce_Paradigm.PNG">

https://wikis.nyu.edu/download/attachments/74681720/WordCount%20MapReduce%20Paradigm.PNG?version=1&modificationDate=1462902481180&api=v2

<div class="alert alert-success alert-dismissable" id="def2">
<a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a>
 Usar divide y vencerás nos ayuda cuando tenemos muchos datos, <strong>MapReduce es necesario cuando tenemos cantidades masivas de datos! La idea es hacer cómputo masivo con hardware standar.</strong>
</div>

Además de Spark hay más sistemas basados en MapReduce como el HDFS de HADOOP.
El sistema de archivos en Spark sí es HDFS, pero Spark usa memoria distribuida en lugar de sólo archivos distribuidos.

## La organización del software de Spark
<img src="./images/software_spark.png" style="width: 70%">
En un clúster **el master controla todo lo que hacen las demás computadoras** (esclavos). Pero **los esclavos hacen todo el trabajo**.

* El **driver** corre en el master.
* Los **RDDs** (la estructura de datos principal de Spark) están particionados a través de los workers.
* Los **workers** administran estas particiones y sus recursos. En general ha un worker por cada nodo esclavo. Se counica con el clúster manager para decirle qué tanto espacio queda y para qué.
* Los ejecutores ejecutan tareas en su partición.Estas tareas llegan de tu programa a través del spark context.
* **Caché** es la memoria que está siendo usada para guardar RDD's (memoria local en el nodo esclavo).

<img src="./images/Software_org_spark_2.png" style="width: 70%">

## El Spark context

Se considera el objeto más importante en Spark
Conecta a Spark con la infraestructura que usa para correr todos los nodos worker.
Sólo se usa uno cada vez que se corre Spark.
Es el camino para comunicarse con el sistema de Spark.

<img src="./images/spark_context.png" style="width: 70%">

## Inicializando el Spark Context


In [ ]:
#Initiallizing Spark (just run once) https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-sparkcontext-creating-instance-internals.html
import pyspark
sc = pyspark.SparkContext('local[*]')
sc